<a href="https://colab.research.google.com/github/fa7e/Calculador-de-Stock/blob/main/Etiquetas_Falabella_WEB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.7 MB/s eta 0:00:00


In [17]:
import pandas as pd
from reportlab.lib.units import cm
from reportlab.pdfgen import canvas
from reportlab.graphics.barcode import code128

# --- Leer archivo Excel ---
df = pd.read_excel("etiquetas.xlsx")  # debe tener columnas: codigo_local, codigo_sublinea, barcode_texto, SKU, unidades

# --- Configuración de página ---
page_width = 10 * cm
page_height = 15 * cm

# --- Crear PDF ---
c = canvas.Canvas("etiquetas.pdf", pagesize=(page_width, page_height))

for idx, row in df.iterrows():
    codigo_local = str(row["codigo_local"])
    codigo_sublinea = str(row["codigo_sublinea"])
    barcode_texto = str(row["barcode_texto"])
    sku = str(row["SKU"])
    unidades = str(row["unidades"])

    # --- Líneas de ejemplo ---
    x_local = page_width - 7*cm
    x_sublinea = page_width - 3*cm
    line_length = 8 * cm

    c.setLineWidth(1)
    c.line(x_local, page_height, x_local, page_height - line_length)
    c.line(x_sublinea, page_height, x_sublinea, page_height - line_length)

    # --- Texto vertical "Local" y "Sub-Linea" ---
    text_below_y_Local = page_height - line_length + 0.8*cm
    text_below_y_sublinea = page_height - line_length + 1.3*cm

    c.saveState()
    c.setFont("Helvetica-Bold", 0.5*cm)
    c.translate(x_local + 0.4*cm, text_below_y_Local)
    c.rotate(90)
    c.drawCentredString(0, 0, "Local:")
    c.restoreState()

    c.saveState()
    c.setFont("Helvetica-Bold", 0.5*cm)
    c.translate(x_sublinea + 0.4*cm, text_below_y_sublinea)
    c.rotate(90)
    c.drawCentredString(0, 0, "Sub-Linea:")
    c.restoreState()

    # --- Código editable SOBRE las líneas (vertical) ---
    start_y = page_height - line_length + 0.5*cm

    # Local
    c.saveState()
    c.setFont("Helvetica", 2*cm)
    c.translate(x_local - 0.1*cm, start_y)
    c.rotate(90)
    c.drawString(0, 0, codigo_local)
    c.restoreState()

    # Sublinea
    c.saveState()
    c.setFont("Helvetica", 2*cm)
    c.translate(x_sublinea - 0.1*cm, start_y)
    c.rotate(90)
    c.drawString(0, 0, codigo_sublinea)
    c.restoreState()

    # --- NUEVO: SKU y Unidades (debajo de Sub-Linea) ---
    sku_y_offset = start_y - 1.5*cm  # 2 cm más abajo del inicio de sublinea
    c.saveState()
    c.setFont("Helvetica", 0.5*cm)
    c.translate(x_sublinea + 2*cm, sku_y_offset)
    c.rotate(90)
    c.drawString(0, 0, f"SKU: {sku} - Unidad: {unidades}")
    c.restoreState()

    # --- Configuración del código de barras ---
    barcode_width = 8 * cm
    barcode_height = 3.5 * cm
    barcode_x = 1 * cm
    barcode_y = 0.5 * cm

    barcode = code128.Code128(
        barcode_texto,
        barHeight=barcode_height,
        barWidth=1.5,
        humanReadable=False
    )
    barcode.drawOn(c, barcode_x, barcode_y)

    # --- Texto numérico encima del código de barras ---
    c.saveState()
    c.setFont("Helvetica", 0.3*cm)
    text_y_position = barcode_y + barcode_height + 0.4*cm
    c.translate(page_width/2, text_y_position)
    c.rotate(180)
    c.drawCentredString(0, 0, barcode_texto)
    c.restoreState()

    # --- Nueva página si no es la última ---
    if idx < len(df) - 1:
        c.showPage()

# --- Guardar PDF ---
c.save()
print("✅ PDF generado: etiquetas.pdf")


✅ PDF generado: etiquetas.pdf
